In [ ]:
# 1. Setup & Install
!pip install timm --quiet
import os
import torch
from torch import nn, optim
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader
import timm


In [ ]:
# 2. Data Preparation
DATA_DIR = "/kaggle/input/asl-alphabet"  # add dataset via Kaggle UI

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Organize dataset
ASL_DIR = "/kaggle/working/asl_images"
for split in ["train", "val"]:
    os.makedirs(f"{ASL_DIR}/{split}", exist_ok=True)

import glob, shutil
classes = os.listdir(f"{DATA_DIR}/asl_alphabet_train/asl_alphabet_train")
for cls in classes:
    imgs = glob.glob(f"{DATA_DIR}/asl_alphabet_train/asl_alphabet_train/{cls}/*.jpg")
    split_idx = int(0.9 * len(imgs))
    for i, img in enumerate(imgs):
        dst = "train" if i < split_idx else "val"
        os.makedirs(f"{ASL_DIR}/{dst}/{cls}", exist_ok=True)
        os.symlink(img, f"{ASL_DIR}/{dst}/{cls}/{os.path.basename(img)}")

In [ ]:
# Load datasets
train_dataset = datasets.ImageFolder(f"{ASL_DIR}/train", transform=transform)
val_dataset = datasets.ImageFolder(f"{ASL_DIR}/val", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4)

print(f"Classes: {train_dataset.classes}")
print(f"Train size: {len(train_dataset)}, Val size: {len(val_dataset)}")

# 3. Load & Modify MobileNetV3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = timm.create_model("mobilenetv3_small_100", pretrained=True)
model.classifier = nn.Linear(model.classifier.in_features, len(train_dataset.classes))
model = model.to(device)


In [ ]:

# 4. Training Setup
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# 5. Training Loop
EPOCHS = 10  # Increase to 30-50 later
for epoch in range(1, EPOCHS + 1):
    model.train()
    total, correct, loss_sum = 0, 0, 0

    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outs = model(imgs)
        loss = criterion(outs, labels)
        loss.backward()
        optimizer.step()

        loss_sum += loss.item() * labels.size(0)
        _, preds = outs.max(1)
        correct += preds.eq(labels).sum().item()
        total += labels.size(0)

    train_acc = correct / total
    train_loss = loss_sum / total

    # Validation
    model.eval()
    total, correct = 0, 0
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outs = model(imgs)
            _, preds = outs.max(1)
            correct += preds.eq(labels).sum().item()
            total += labels.size(0)
    val_acc = correct / total

    print(f"Epoch {epoch}/{EPOCHS}: Train Loss: {train_loss:.4f}, "
          f"Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.0 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 22.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.2 MB/s eta 0:00:00:00:0100:01
Classes: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 

model.safetensors:   0%|          | 0.00/10.2M [00:00<?, ?B/s]

Epoch 1/10: Train Loss: 0.1636, Train Acc: 0.9605, Val Acc: 0.9895
Epoch 2/10: Train Loss: 0.0096, Train Acc: 0.9970, Val Acc: 0.9970
Epoch 3/10: Train Loss: 0.0102, Train Acc: 0.9971, Val Acc: 0.9402
Epoch 4/10: Train Loss: 0.0076, Train Acc: 0.9980, Val Acc: 0.9961
Epoch 5/10: Train Loss: 0.0061, Train Acc: 0.9983, Val Acc: 0.9992
Epoch 6/10: Train Loss: 0.0071, Train Acc: 0.9979, Val Acc: 0.9999
Epoch 7/10: Train Loss: 0.0053, Train Acc: 0.9985, Val Acc: 0.9993
Epoch 8/10: Train Loss: 0.0044, Train Acc: 0.9987, Val Acc: 0.9991
Epoch 9/10: Train Loss: 0.0028, Train Acc: 0.9993, Val Acc: 0.9998
Epoch 10/10: Train Loss: 0.0051, Train Acc: 0.9984, Val Acc: 0.9960
✅ Model saved as mobilenetv3_asl.pth


In [ ]:
# 6. Save Model
torch.save(model.state_dict(), "mobilenetv3_asl.pth")

In [ ]:
with open("labels.txt", "w") as f:
    for cls in train_dataset.classes:
        f.write(f"{cls}\n")


✅ labels.txt file created with all class names
